# Glosario Total Renting

In [1]:
# Cargamos librerias

import pandas as pd
import people_also_ask
from youtubesearchpython import VideosSearch
import re
import time
from numpy import random

In [2]:
# Improtamos en un DF los coches para la búsqueda

coches = pd.read_csv('https://raw.githubusercontent.com/mariopaj/Trabajo_TR/main/coches.csv') # Introducir ruta en donde esté ubicado el csv de las Key Words
  
coches = list(coches.Keyword.apply(lambda x: x.strip())) # Lo pasamos a una lista y eliminamos espacios

coches[:10]


['Abarth 500',
 'Alfa Romeo Giulia',
 'Alfa Romeo Stelvio',
 'Audi A1',
 'Audi A3',
 'Audi A4',
 'Audi A5',
 'Audi A6',
 'Audi A7',
 'Audi A8']

In [3]:
len(coches)

255

In [4]:
# Función que devuelve pregunta-respuesta en formato HTML

def obtener_preguntas_frecuentes_html(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    string
        devuelve un string con las preguntas y las respuestas en formato HTML
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        contenido = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']) + '</p>')

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) + '<br>' for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + '<br>' + values
                    answers.append(result)  
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + result + '</p>')

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

        contenido = [''.join(contenido)][0]

        return contenido
    
    except:
        return print('There are no frequently asked questions')

In [5]:
# Función que devuelve pregunta-respuesta en formato Lista

def obtener_preguntas_frecuentes(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    df
        devuelve un df de preguntas y respuestas
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + values
                    answers.append(result)

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

            # Creamos una lista sólo de las preguntas que tienen respuesta de cada KeyWord
            question_answer = []
            for question, answer in zip(questions, answers):
                if answer != '':
                    question_answer.append([question, answer])
                else:
                    pass
            
            # Desglosamos la lista
            flat_list = []
            for q in question_answer:
                for item in q:
                    flat_list.append(item)
            
            flat_list.insert(0, key_word)
        
        return flat_list
    
    except:
        return print('There are no frequently asked questions')

In [12]:
# Sacamos el Excel de las preguntas y respuestas frecuentes

def preguntas_frecuentes_and_video_to_excel(kws, n=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word y almacenarlas en un excel
    
    Parameters
    ----------
    kws: list
        lista de keywords de las que queremos sacar las preguntas frecuentes
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    df
        devuelve un df de preguntas y respuestas
    '''

    dffq = pd.DataFrame()
    for kw in kws:
        fq = obtener_preguntas_frecuentes(kw, n, search) # Sacamos la lista de pregunta-respuesta
        
        try:
            fq.insert(1, VideosSearch(kw, limit = 1).result()['result'][0]['link']) # insertamos el video en la segunda columna
        except:
            fq.insert(1, '')
            
        df = pd.DataFrame(fq).transpose()
        dffq = pd.concat([dffq, df])
        dffq.to_excel('glosario.xlsx', index = False) # Vamos almacenando los resultados en un excel (marcar la ruta donde quieres que se ubique el excel)
        
        print(kw)
        
        time.sleep(random.uniform(1, 2)) # Time sleep random entre 1 y 2
        
    return dffq.head()

In [14]:
preguntas_frecuentes_and_video_to_excel(coches[:2], 5, search = 'Search') # Prueba para varios coches

Abarth 500
Alfa Romeo Giulia


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Abarth 500,https://www.youtube.com/watch?v=q2zRmgjry5w,Who makes the Abarth?,Fiat Chrysler Automobiles,How much does an Abarth 500 cost?,Used 2019 FIAT 500 Abarth Pricing The Abarths ...,How fast is the Fiat Abarth?,127 mph,Does Abarth have a Ferrari engine?,"Abarth, the high-performance division of Fiat,...",Is Abarth made by Ferrari?,Abarth says the car has been produced because ...,Is the Fiat 500 Abarth fast?,The best version of the Abarth 500 has a top s...
0,Alfa Romeo Giulia,https://www.youtube.com/watch?v=qepFQDxES70,What is wrong with Alfa Romeo Giulia?,Consumer Reports thinks the Alfa Romeo Giulia ...,Are Alfa Romeos made by Ferrari?,"Ferrari doesn't own Alfa Romeo; in fact, they ...",Is the Alfa Romeo Giulia engine from Ferrari?,With its rhapsodic twin-turbo 2.9-liter V-6 an...,Is Alfa Romeo Giulia a good car?,"Is the Alfa Romeo Giulia a Good Car? Yes, the ...",Does the Alfa Romeo Giulia have a Ferrari engine?,The class-leading performance numbers of the A...,NaN,NaN


In [15]:
# Traducimos el excel glosario a español

def traducir_glosario(route : str, language = 'ES'):
    
    '''
    Esta función sirve para traducir el glosario en formato excel al idioma que queramos
    
    Parameters
    ----------
    route: str
        Ruta en la que tenemos ubicado el excel glosario
    
    language: str
        Idioma al que queremos traducir el glosario
        
    
    Returns
    -------
    df
        devuelve el df traducido
    '''
    
    import deepl

    translator = deepl.Translator(str(input('Introduce la clave de autentificación de la API:'))) # Introducimos la clave de autentificacion para poder acceder a las traducciones

    df_original = pd.read_excel(route) # Importamos el excel escrapeado

    df_traduction = pd.DataFrame() # Creamos un DataFrame vacío
    df_traduction[0] = df_original[0] # Añadimos al DataFrame la KeyWord
    df_traduction[1] = df_original[1] # Añadimos al DataFrame el link del video

    for column in range(2, df_original.shape[1]):
        print(column)
        traduction = []
        for sentence in df_original[column]:
            try:
                traduction.append(translator.translate_text(sentence, target_lang=language).text)
            except:
                traduction.append('')

        df_traduction[column] = traduction
        df_traduction.to_excel('glosario_traducido.xlsx', index = False)

    return df_traduction

In [16]:
traducir_glosario('glosario.xlsx')

Introduce la clave de autentificación de la API: d2faa701-e799-87b0-5e80-a79f478a9df2:fx


2
3
4
5
6
7
8
9
10
11
12
13


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Abarth 500,https://www.youtube.com/watch?v=q2zRmgjry5w,¿Quién fabrica el Abarth?,Fiat Chrysler Automobiles,¿Cuánto cuesta un Abarth 500?,Precios del FIAT 500 Abarth usado 2019 Los Aba...,¿Cómo de rápido es el Fiat Abarth?,127 mph,¿Tiene Abarth un motor Ferrari?,"¡Abarth, la división de alto rendimiento de Fi...",¿Abarth está fabricado por Ferrari?,Abarth afirma que el coche se ha producido gra...,¿Es rápido el Fiat 500 Abarth?,La mejor versión del Abarth 500 alcanza una ve...
1,Alfa Romeo Giulia,https://www.youtube.com/watch?v=qepFQDxES70,¿Qué pasa con el Alfa Romeo Giulia?,Consumer Reports cree que el Alfa Romeo Giulia...,¿Los Alfa Romeos son fabricados por Ferrari?,Ferrari no es propietaria de Alfa Romeo; de he...,¿El motor del Alfa Romeo Giulia es de Ferrari?,"Con su rapsoda V-6 de 2,9 litros y una transmi...",¿Es el Alfa Romeo Giulia un buen coche?,"¿Es el Alfa Romeo Giulia un buen coche? Sí, el...",¿Tiene el Alfa Romeo Giulia un motor Ferrari?,Las cifras de rendimiento del Alfa Romeo Giuli...,,
